In [10]:
import numpy as np
import cvxpy as cp

### Mosek installation test

In [11]:
x = cp.Variable(2)
obj = cp.Minimize(x[0] + cp.norm(x, 1))
constraints = [x >= 2]
prob = cp.Problem(obj, constraints)

prob.solve(solver=cp.MOSEK)
print("optimal value with MOSEK:", prob.value)

optimal value with MOSEK: 6.0


In [12]:
print(cp.installed_solvers())

['CLARABEL', 'ECOS', 'ECOS_BB', 'MOSEK', 'OSQP', 'SCIPY', 'SCS']


### LMI Hello World

This is checking if there is a matrix P for the Lyapunov Quadratic Equation for A

In [13]:
eps = 1e-9 # 

A = np.array([[-0.3937, 0.4057], [0.1763, -0.1964]])
n = A.shape[0]

P = cp.Variable((n, n), symmetric=True)
Q = np.eye(n,n)
constraints = [P >= eps]
constraints += [A.T @ P + P @ A + Q <= -eps]

prob = cp.Problem(cp.Minimize(None), constraints=constraints)
prob.solve(solver=cp.MOSEK)

nan

In [14]:
is_positive_defined = 'positive defined' if all(np.linalg.eig(P.value).eigenvalues > 0) else 'NOT positive definite'

print(f"The solution is \'{prob.status}\', with eig(P) = {np.linalg.eig(P.value).eigenvalues} -> P is {is_positive_defined}")

The solution is 'optimal', with eig(P) = [ 0.85993742 57.72109604] -> P is positive defined


### System Controller

In [15]:
A_control = np.array([
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [-3, 1, 2, 3],
    [2, 1, 0, 0]
])

B_control = np.array([
    [0, 0],
    [0, 0],
    [1, 2],
    [0, 2]
])

n_control = A_control.shape[0]

C = np.concat([B_control, np.linalg.matrix_power(A_control, 1) @ B_control, np.linalg.matrix_power(A_control, 2) @ B_control, np.linalg.matrix_power(A_control, 3) @ B_control], axis=1)
full_rank = 'has' if np.linalg.matrix_rank(C) == A_control.shape[0] else 'does not have'

print(f"the controllability matrix {full_rank} full rank ")

the controllability matrix has full rank 


#### Checking Feasibility

In [16]:
P_feas = cp.Variable((n_control, n_control), symmetric=True)

constraints_feas = [P_feas >= eps]
constraints_feas += [A_control.T @ P_feas + P_feas @ A_control <= -eps]

prob_feas = cp.Problem(cp.Minimize(None), constraints=constraints_feas)
prob_feas.solve(solver=cp.MOSEK)

nan

In [17]:
is_positive_defined = 'positive defined' if all(np.linalg.eig(P_feas.value).eigenvalues > 0) else 'NOT positive defined'
print(f'P = \n{P_feas.value}')
print(f"The solution is \'{prob_feas.status}\', with eig(P) = {np.linalg.eig(P_feas.value).eigenvalues} -> P is {is_positive_defined}")

P = 
[[1.e-09 1.e-09 1.e-09 1.e-09]
 [1.e-09 1.e-09 1.e-09 1.e-09]
 [1.e-09 1.e-09 1.e-09 1.e-09]
 [1.e-09 1.e-09 1.e-09 1.e-09]]
The solution is 'optimal', with eig(P) = [ 4.00000000e-09  8.31158914e-26 -8.03560591e-41  0.00000000e+00] -> P is NOT positive defined


#### Controller Design

In [18]:
Q_control = cp.Variable((n_control, n_control), symmetric=True)
Z_control = cp.Variable((B_control.shape[1], n_control))

constraints_control = [ Q_control >= eps]
constraints_control += [ Q_control @ A_control.T + Z_control.T @ B_control.T + A_control @ Q_control + B_control @ Z_control <= -eps]

prob_control = cp.Problem(cp.Minimize(None), constraints=constraints_control)
prob_control.solve(solver=cp.MOSEK)

nan

In [19]:
P_control = Q_control.T.value
K = (Z_control @ P_control).value

is_positive_defined = 'positive defined' if all(np.linalg.eig(P_control).eigenvalues > 0) else 'NOT positive definite'
print(f'P = \n{P_control}')
print(f"The solution is \'{prob_control.status}\', with eig(P) = {np.linalg.eig(P_control).eigenvalues} -> P is {is_positive_defined}")
print(f"The designed controller K = \n{K}")

P = 
[[1.e-09 1.e-09 1.e-09 1.e-09]
 [1.e-09 1.e-09 1.e-09 1.e-09]
 [1.e-09 1.e-09 1.e-09 1.e-09]
 [1.e-09 1.e-09 1.e-09 1.e-09]]
The solution is 'optimal', with eig(P) = [ 4.00000000e-09  8.31158914e-26 -8.03560591e-41  0.00000000e+00] -> P is NOT positive definite
The designed controller K = 
[[ 4.13590306e-34  4.13590306e-34  4.13590306e-34  4.13590306e-34]
 [-8.50000000e-18 -8.50000000e-18 -8.50000000e-18 -8.50000000e-18]]
